In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time 

In [2]:
url= 'https://www.qiyas.sa/en/Statistics-Data/Schools/Pages/DetResults.aspx'

#Create a new Safari browser object
driver = webdriver.Safari()
driver.implicitly_wait(100)

#Go to the website
driver.get(url)

##Extract data for Achievement, Girls, Scientific, Private##
    
#Click on 'Achievement'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[1]/div/table/tbody/tr[1]/td/div/div[2]/div/label').click()

#Click on 'Girls'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[1]/div/table/tbody/tr[2]/td/div/div[2]/div/label').click() 

#Click on 'Scientific'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[1]/div/table/tbody/tr[3]/td/div/div[1]').click() 

#Click on 'Private'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[1]/div/table/tbody/tr[4]/td/div/div[2]/div').click() 

#Click on 'View the order of all schools'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[1]/div/table/tbody/tr[5]/td/span[1]/label').click() 

#Click on 'Submit'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[2]/div').click()

In [3]:
#Retrieving table from first page 
time.sleep(5)
soup_cbsp= BeautifulSoup(driver.page_source, 'html.parser')

#Parse and extract data 
table= soup_cbsp.find_all(class_='table-responsive')[1]

rows=table.find_all('tr')

COLUMNS= ['General Rank', 'Ministerial Number', 'School Name', 
          'City', 'Region', 'School Type', 'Category', 'Average']

sub_df=[]
for row in rows:
    cells= row.select('td')
    cells=cells[0:8]
    sub_df.append([cell.text.strip() for cell in cells])

data_agsp = pd.DataFrame(sub_df, columns=COLUMNS).drop([0,1], axis=0).reset_index(drop=True)
data_agsp

,General Rank,Ministerial Number,School Name,City,Region,School Type,Category,Average
0,1,55915,ثانوية مدارس الظهران الاهلية (مقررات),الخبر,AlSharquia,Domestic,مقررات,92
1,2,55919,ثانوية مدارس الجامعة الاهلية للبنات,الخبر,AlSharquia,Domestic,تعليم عام بنات,87.51
2,3,422495,مدارس منارات المدينة المنورة الأهلية - المرحلة...,المدينة المنورة,Madina,Domestic,مقررات,87.46
3,4,155010,مدارس السلام الخاصة للبنات - نظام المقررات,الخبر,AlSharquia,Domestic,مقررات,87.24
4,5,55921,ثانوية مدارس العزيزيه الاهلية للبنات,الخبر,AlSharquia,Domestic,تعليم عام بنات,86.72
5,6,96623,ثانوية الفهد الأهلية,الرياض,Riyadh,Domestic,تعليم عام بنات,86.26
6,7,35066,ثانوية الذكر بجدة نظام مقررات,جدة,Makkah,Domestic,مقررات,86.19
7,8,59009,ثانوية الانجال الأهلية,الأحساء,AlSharquia,Domestic,تعليم عام بنات,86.13
8,12,35042,ثانوية الزهراء بجدة نظام مقررات,جدة,Makkah,Domestic,مقررات,85.23
9,13,56201,ثانوية مدارس البسام الاهلية للبنات -مقررات,الدمام,AlSharquia,Domestic,مقررات,85.07


In [4]:
while True:
    try:
        next_page_btn= driver.find_elements(By.CLASS_NAME, "Next")
        if not next_page_btn:
            print("No next button")
            break
        next_btn= next_page_btn[0]
        if next_btn.get_attribute("class")== "aspNetDisabled Next":
            print("No more pages left")
            break
        else:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "Next"))).click()
        
            time.sleep(10)
       
            soup_pgs= BeautifulSoup(driver.page_source, 'html.parser')
        
            #Parse and extract data 
            table_pgs= soup_pgs.find_all(class_='table-responsive')[1]
        
            rows_pgs=table_pgs.find_all('tr')
        
            COLUMNS_pgs= ['General Rank', 'Ministerial Number', 'School Name', 
              'City', 'Region', 'School Type', 'Category', 'Average']
        
            sub_df_pgs=[]
            for row_pgs in rows_pgs:
                cells_pgs= row_pgs.select('td')
                cells_pgs=cells_pgs[0:8]
                sub_df_pgs.append([cell_pgs.text.strip() for cell_pgs in cells_pgs])
        
            data_pgs = pd.DataFrame(sub_df_pgs, columns=COLUMNS_pgs).drop([0,1], axis=0).reset_index(drop=True)
            data_agsp=data_agsp.append(data_pgs, ignore_index=True)
    except TimeoutException:
        time.sleep(10)
        print("Exception has been thrown. No more pages left")
        break
driver.quit()

No more pages left


In [5]:
data_agsp

,General Rank,Ministerial Number,School Name,City,Region,School Type,Category,Average
0,1,55915,ثانوية مدارس الظهران الاهلية (مقررات),الخبر,AlSharquia,Domestic,مقررات,92
1,2,55919,ثانوية مدارس الجامعة الاهلية للبنات,الخبر,AlSharquia,Domestic,تعليم عام بنات,87.51
2,3,422495,مدارس منارات المدينة المنورة الأهلية - المرحلة...,المدينة المنورة,Madina,Domestic,مقررات,87.46
3,4,155010,مدارس السلام الخاصة للبنات - نظام المقررات,الخبر,AlSharquia,Domestic,مقررات,87.24
4,5,55921,ثانوية مدارس العزيزيه الاهلية للبنات,الخبر,AlSharquia,Domestic,تعليم عام بنات,86.72
5,6,96623,ثانوية الفهد الأهلية,الرياض,Riyadh,Domestic,تعليم عام بنات,86.26
6,7,35066,ثانوية الذكر بجدة نظام مقررات,جدة,Makkah,Domestic,مقررات,86.19
7,8,59009,ثانوية الانجال الأهلية,الأحساء,AlSharquia,Domestic,تعليم عام بنات,86.13
8,12,35042,ثانوية الزهراء بجدة نظام مقررات,جدة,Makkah,Domestic,مقررات,85.23
9,13,56201,ثانوية مدارس البسام الاهلية للبنات -مقررات,الدمام,AlSharquia,Domestic,مقررات,85.07


In [6]:
data_agsp.to_csv('ach_girls_sci_priv.csv', index=False)